# CNN implementation on Dementia Dataset

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.model_selection import train_test_split
import cv2 as cv
import os, sys

In [3]:
path1=''
path2=''
files1=os.listdir(path1)
files2=os.listdir(path2)
Num_files_N=len(files1)
Num_files_D=len(files2)
dataset_len=Num_files_N+Num_files_D

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
Num_files_N

In [ ]:
name=path1+'/'+files1[0]
img=cv.imread(name)

In [ ]:
plt.imshow(img,cmap='gray')

In [ ]:
img.shape

# Dataset Creation

For every file

1. Read the image
2. Convert it to grayscale (Optional)
3. Resize to (100,100)
4. Preprocessing: Normalization
5. Reshape to (100,100,1)
6. Create the dataset

In [ ]:
data=np.zeros((dataset_len,100,100,3))
label=[]
data.shape

In [8]:
for i in range(Num_files_N):
    name=path1+'/'+files1[i]
    img=cv.imread(name)
    #img_gs=cv.cvtColor(img,cv.COLOR_RGB2GRAY)
    img=cv.resize(img,(100,100))
    img=img/255
    #img_gs=img_gs.reshape(100,100,3)
    data[i,:,:]=img
    label.append('Normal')


In [ ]:
data[99].ndim

In [ ]:
plt.imshow(data[99])
#plt.imshow(cv.resize(data[0],(100,100)),cmap='gray')

In [9]:
for i in range(Num_files_D):
    name=path2+'/'+files2[i]
    img=cv.imread(name)
    img=cv.resize(img,(100,100))
    img=img/255
    data[i+Num_files_N,:,:]=img
    label.append('Dementia')


In [ ]:
#plt.imshow(data[100],cmap='gray')
plt.imshow(cv.resize(data[100],(100,100)),cmap='gray')

In [ ]:
label[100]

Label Encoding

In [11]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
lab=le.fit_transform(label)

In [ ]:
type(label)

In [ ]:
type(lab)

Train and test dataset spilt

In [12]:
train_images,test_images,train_labels,test_labels=train_test_split(data,lab,test_size=0.2, random_state=1)

In [ ]:
train_labels

In [ ]:
print('Train Dataset Size:',np.size(train_labels))
print('Test Dataset Size:',np.size(test_labels))

In [ ]:
np.unique(test_labels,return_counts=True)

# Define the CNN architecture

Create the convolutional base

1. Convolutional : 32 filters 3x3
2. Maxpooling: 2x2
3. Convolutional : 64 filters 5x5
4. Convolutional : 32 filters 3x3
5. Maxpooling: 3x3


In [13]:
network=models.Sequential()
network.add(layers.Conv2D(32,(3,3),activation='relu', input_shape=(100,100,3)))
network.add(layers.MaxPooling2D((2,2)))
network.add(layers.Conv2D(64,(7,7),activation='relu'))
network.add(layers.Conv2D(32,(3,3),activation='relu'))
network.add(layers.MaxPooling2D((3,3)))

Check summary of convolutional base

In [ ]:
network.summary()

Build the classifier on top of the convolutional base

In [15]:
network.add(layers.Flatten())
network.add(layers.Dense(80,activation='relu'))
network.add(layers.Dense(50,activation='relu'))
network.add(layers.Dense(2,activation='softmax'))

In [ ]:
network.summary()

Compile and train the model

In [17]:
network.compile(optimizer='sgd',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
trained_network=network.fit(train_images,train_labels,epochs=30,validation_data=(test_images,test_labels))

In [ ]:
plt.plot(trained_network.history['accuracy'],label='Training Accuracy')
plt.plot(trained_network.history['val_accuracy'],label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0,1.1])
plt.legend(loc='lower right')

In [ ]:
plt.plot(trained_network.history['loss'],label='Training Loss')
plt.plot(trained_network.history['val_loss'],label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')

Evaluate the network

In [ ]:
test_loss,test_acc=network.evaluate(test_images,test_labels)

In [ ]:
y_predict=network.predict(test_images)

In [ ]:
y_pred=[]
for val in y_predict:
    y_pred.append(np.argmax(val))

In [ ]:
print(metrics.confusion_matrix(test_labels,y_pred))

[[17  0]
 [23  0]]


In [ ]:
print(metrics.accuracy_score(test_labels,y_pred))

0.425


In [ ]:
for layer in network.layers:
    if 'conv' in layer.name:
        weights, bias= layer.get_weights()
        #print(layer.name, filters.shape)

        #normalize filter values between  0 and 1 for visualization
        f_min, f_max = weights.min(), weights.max()
        filters = (weights - f_min) / (f_max - f_min)
        print(filters.shape[3])
        filter_cnt=1

        #plotting all the filters
        for i in range(filters.shape[3]):
            #get the filters
            filt=filters[:,:,:, i]
            #plotting each of the channel, color image RGB channels
            for j in range(filters.shape[0]):
                ax= plt.subplot(filters.shape[3], filters.shape[0], filter_cnt  )
                ax.set_xticks([])
                ax.set_yticks([])
                plt.imshow(filt[:,:, 1])
                filter_cnt+=1
        plt.show()

In [24]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img

In [ ]:
img_path='' #dementia
# Define a new Model, Input= image
# Output= intermediate representations for all layers in the
# previous model after the first.
successive_outputs = [layer.output for layer in network.layers[1:]]
#visualization_model = Model(img_input, successive_outputs)
visualization_model = tf.keras.models.Model(inputs = network.input,
                                            outputs = successive_outputs)
#Load the input image
img = load_img(img_path, target_size=(100, 100))
# Convert ht image to Array of dimension (100,100,3)
x   = img_to_array(img)
x   = x.reshape((1,) + x.shape)
# Rescale by 1/255
x /= 255.0
# Let's run input image through our vislauization network
# to obtain all intermediate representations for the image.
successive_feature_maps = visualization_model.predict(x)
# Retrieve are the names of the layers, so can have them as part of our plot
layer_names = [layer.name for layer in network.layers]
for layer_name, feature_map in zip(layer_names, successive_feature_maps):
  print(feature_map.shape)
  if len(feature_map.shape) == 4:

    # Plot Feature maps for the conv / maxpool layers, not the fully-connected layers

    n_features = feature_map.shape[-1]  # number of features in the feature map
    size       = feature_map.shape[ 1]  # feature map shape (1, size, size, n_features)

    # We will tile our images in this matrix
    display_grid = np.zeros((size, size * n_features))

    # Postprocess the feature to be visually palatable
    for i in range(n_features):
      x  = feature_map[0, :, :, i]
      x -= x.mean()
      x /= x.std ()
      x *=  64
      x += 128
      x  = np.clip(x, 0, 255).astype('uint8')
      # Tile each filter into a horizontal grid
      display_grid[:, i * size : (i + 1) * size] = x
# Display the grid
    scale = 20. / n_features
    plt.figure( figsize=(scale * n_features, scale) )
    plt.title ( layer_name )
    plt.grid  ( False )
    plt.imshow( display_grid, aspect='auto', cmap='viridis' )